# グローバーのアルゴリズム

グローバーのアルゴリズムとは、整列化されていないデータベースから特定のデータを探索するための量子アルゴリズムである。この章ではまずアルゴリズムの数学的なメカニズム、次に我々が開発している量子シミュレータであるQulacsによる実装例を提示する。

アルゴリズムの大まかな手順は以下の通りである。

## 1. 重ね合わせ状態を生成する。
### 数学的原理
このアルゴリズムは、全てのデータに番号をつけ、各データが観測される確率振幅が同じ状態(重ね合わせ状態)を生成する。具体的にはデータの数を$N$個とすると$$|\psi\rangle = \frac{1}{N^{1/2}}\sum_{x=0}^{N-1} |x\rangle$$と表される。

### 量子ゲート　
この状態を生成するのに必要なのが、Hゲート(アダマールゲート)と呼ばれる量子ゲートである。

$
 H = \left(            
    \begin{array}{ll}   
      1 & 1  \\
      1 & -1  \\
    \end{array}
  \right) 
$
  
例えば、$H|0\rangle = \left( \begin{array}{cc} 1 & 1\\ 1 & -1\\ \end{array} \right) \left(\begin{array}{c} 1 \\ 0 \\ \end{array} \right)  = \left( \begin{array}{c} 1\\ 1\\ \end{array} \right) = \frac{|0\rangle + |1\rangle}{\sqrt{2}}= \frac{1}{2^{1/2}}\sum_{x=0}^{1} |x\rangle$ 
以上のように、１つの状態にHゲートをかけると$|0\rangle$,$|1\rangle$両方含む状態を生成することができる。

例えば、データが4つ(2量子ビットで表現可能)の場合、重ね合わせ状態は以下の要領で表せる。

$H^{\otimes 2}|0\rangle|0\rangle = \frac{|0\rangle+|1\rangle}{\sqrt{2}} \bullet \frac{|0\rangle+|1\rangle}{\sqrt{2}} = \frac{|00\rangle+|01\rangle+|10\rangle+|11\rangle}{2} = \frac{1}{4^{1/2}}\sum_{x=0}^{3} |x\rangle$

## 2.探索したいデータの確率振幅を反転させる。 

### 数学的原理
一般的に量子アルゴリズムの過程において、答えを識別するブラックボックスの部分をオラクルと呼ぶ。[2]今回は重ね合わせ状態から特定の状態を取り出す時に必要な計算(実装の場合は量子回路)を指す。
まず、求めたい状態を$|\psi\rangle$とし、以下の条件を満たす関数$f(x)$を考える。　$$f(x) = \begin{eqnarray}
\left\{
\begin{array}{l}
1(x = |\psi\rangle)\\
0(x \neq |\psi\rangle)
\end{array}
\right.
\end{eqnarray}$$

実装するオラクル$U$は以下の機能を持つものとする。$|x\rangle\xrightarrow{U}(-1)^{f(x)}|x\rangle$　　

そして、このオラクルは $ U =  I - 2|\psi\rangle\langle\psi| $で表される。 [3]

$U|\psi\rangle = I|\psi\rangle - 2|\psi\rangle\langle\psi||\psi\rangle = I|\psi\rangle - 2|\psi\rangle = -|\psi\rangle $ 

$U|\phi\rangle = I|\phi\rangle - 2|\psi\rangle\langle\psi||\phi\rangle = I|\phi\rangle = |\phi\rangle $

$(|\phi\rangleは求めたくない状態)$

これを視覚的に表現すると以下のようになる。(軸の説明は後述する) 
![screen shot 2019-02-27 at 2 57 26 pm](https://user-images.githubusercontent.com/45162150/53469100-0ab01100-3aa0-11e9-866c-6581110853fc.png)






## 3. 拡散変換を施す
### 数学的原理 
データベース内のデータ数を$M$、求めるデータ数を$N$とし、求めるべきではないものが求まる確率の和を$|\alpha\rangle$、求まるべきものが求まる確率の和を$|\beta\rangle$とする。これらは以下のように表される。[5]　　

$$|\alpha\rangle = \frac{1}{\sqrt{N-M}}\sum_{x}^{} |x\rangle$$
$$|\beta\rangle = \frac{1}{\sqrt{M}}\sum_{x}^{} |x'\rangle$$  

これらより、量子状態$|\psi\rangle$は$|\psi\rangle = \frac{N-M}{N}|\alpha\rangle + \frac{M}{N}|\beta\rangle$と表せる。
$\cos{\frac{\theta}{2}} = \frac{N-M}{N}$を満たす角$\theta$について　

演算子$ G = \left[\begin{array}{rr}\cos\theta&\sin\theta \\　\sin\theta & \cos\theta \end{array}\right]$を考えると、　$G|\psi\rangle = \cos{\frac{3\theta}{2}}|\alpha\rangle + \sin{\frac{3\theta}{2}}|\beta\rangle$となる。　

演算子Gを任意の回数(k回とする)掛けていくと、$G^{k}|\psi\rangle = \cos{\frac{k\theta}{2}}|\alpha\rangle + \sin{\frac{k\theta}{2}}|\beta\rangle$となる。これを視覚的に表現したのが以下の図である。　元の状態(紫色の矢印)がG演算子を何回もかけられたことによって、新たな状態(水色の矢印)になって$|\beta\rangle$軸に近づいていく。つまり、求められるべきものが求まる確率が増加していく。

![screen shot 2019-02-27 at 3 35 48 pm](https://user-images.githubusercontent.com/45162150/53470575-67fa9100-3aa5-11e9-9bc3-6407fe2f22a4.png)

ちなみに、Gを求めて確率が上昇する回数は、$\frac{\pi\sqrt{N}}{4}$回である。(Nは量子ビットの数)[5]

### 量子ゲート
負の値も含めた各状態の確率振幅の平均値を$average$とおくと、$average$から各確率振幅を引き、その差を$average$に足す。この演算を拡散変換という。
2量子ビットで|11>を反転させた場合を例として考える。 

この時、 $average = (\frac{1}{2}+\frac{1}{2}+\frac{1}{2}-\frac{1}{2})*\frac{1}{4} = \frac{1}{4}$

ここで、拡散変換を行うと、各状態を得る確率は以下のようになる。

$P(|00\rangle) = average - \frac{1}{2} + average = \frac{1}{4} - \frac{1}{2} + \frac{1}{4} = 0$

$P(|01\rangle) = average - \frac{1}{2} + average = \frac{1}{4} - \frac{1}{2} + \frac{1}{4} = 0$

$P(|10\rangle) = average - \frac{1}{2} + average = \frac{1}{4} - \frac{1}{2} + \frac{1}{4} = 0$

$P(|11\rangle) = average - (-\frac{1}{2}) + average = \frac{1}{4} + \frac{1}{2} + \frac{1}{4} = 1$

よって、求めたい状態$|11\rangle$を得る確率は 100%になった。

# Qulacsを用いた実装

次に我々が開発している量子シミュレータQulacsを用いた実装例を紹介する。と3量子ビット(|111>)での探索である。なお、Qulacsの詳しい文法に関してはdocumentation ( http://qulacs.org/index.html ) を参照して頂きたい。


### 量子ゲート
1.の後に求めたい状態に応じて以下のオラクルを実装すると、状態の位相を反転させられる。[4]

ちなみに、![screen shot 2019-02-07 at 3 05 25 pm](https://user-images.githubusercontent.com/45162150/52393534-ce654280-2ae9-11e9-9334-601c84af6df1.png)
はCZゲート、
![screen shot 2019-02-07 at 3 06 41 pm](https://user-images.githubusercontent.com/45162150/52393570-fbb1f080-2ae9-11e9-978a-194b3949c7f8.png)
はCCXゲートと呼ばれる量子ゲートである。なお、3量子ビット上の実装に必要なCCZゲートはCCXゲートの1番下のビットをHゲートで挟むことで実装できるが、実装例では他の方法(CZゲートにコントロールビットを１つ追加する)で実装している。

### 2量子ビット
今回まずは2量子ビット(|11>)の例をお見せするが、他の例をご自身で試してみるとより理解が深まると考えられる。　

$ |00\rangle $　
![screen shot 2019-02-07 at 2 38 37 pm](https://user-images.githubusercontent.com/45162150/52392703-62350f80-2ae6-11e9-9170-e7c695760bff.png)
$ |01\rangle $
![screen shot 2019-02-07 at 2 42 15 pm](https://user-images.githubusercontent.com/45162150/52392753-990b2580-2ae6-11e9-8891-0bb41cb0af48.png)
$ |10\rangle $
![screen shot 2019-02-07 at 2 43 09 pm](https://user-images.githubusercontent.com/45162150/52392776-b7712100-2ae6-11e9-9244-ed96c71647ba.png)
$ |11\rangle $ 
![screen shot 2019-02-07 at 2 44 13 pm](https://user-images.githubusercontent.com/45162150/52392813-dd96c100-2ae6-11e9-8203-e10cd83c1104.png)
 
 


In [66]:
#必要なモジュールのインポート
from qulacs import QuantumState,QuantumCircuit

#必要な量子ビットの数
n = 2

#2量子ビットの状態を定義する
state2 = QuantumState(n)

#量子状態を初期化する
state2.set_zero_state()

#2量子ビットの量子回路を生成
circuit = QuantumCircuit(n)

#各量子ビットにHゲートをかけて重ね合わせ状態を生成する
for i in range(n):
    circuit.add_H_gate(i)

#これらのゲートを回路および状態に適用する。
circuit.update_quantum_state(state2)

#状態ベクトル(確率振幅)の取得
print(state2.get_vector())

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]


これらのベクトルは$|00\rangle$,$\,|01\rangle$,$\,|10\rangle$,$\,|11\rangle$の順に並んでいる。
ここで、各状態の確率振幅が等しいことが確認できた。次に取得したい状態($|11\rangle$)の確率振幅の位相を反転させる。

In [22]:
n = 2
state2 = QuantumState(n)
state2.set_zero_state()

circuit = QuantumCircuit(n)

for i in range(n):
    circuit.add_H_gate(i)

#ここからnew
#CZゲート(1番目のビットの値が1の時、2番目の量子ビットにZゲートをかける2量子ゲート)をかける
circuit.add_gate(CZ(0,1))


circuit.update_quantum_state(state2)

print(state2.get_vector())

[ 0.5+0.j  0.5+0.j  0.5+0.j -0.5-0.j]


ここで|11>の確率振幅の位相のみ反転していることが確認できた。
最後に拡散変換を施す。

In [33]:
n = 2
state2 = QuantumState(n)
state2.set_zero_state()

circuit = QuantumCircuit(n)

for i in range(n):
    circuit.add_H_gate(i)
    
circuit.add_gate(CZ(0,1))

#ここからnew
#拡散変換
for i in range(n):
    circuit.add_H_gate(i)
    circuit.add_X_gate(i)

circuit.add_gate(CZ(0,1))

for i in range(n):
    circuit.add_X_gate(i)
    circuit.add_H_gate(i)
    
circuit.update_quantum_state(state2)

print(state2.get_vector())

[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]


これより、$|11\rangle$の確率振幅の絶対値が1(得られる確率が100%)になったので、測定した時に確実に$|11\rangle$が得られる。

### 3量子ビット

今回は$|111\rangle$の例を掲載するが、以下の回路を実装すれば好きな状態を求められる。　

$ |000\rangle$
![screen shot 2019-02-07 at 2 55 09 pm](https://user-images.githubusercontent.com/45162150/52393188-66fac300-2ae8-11e9-84ab-cf15ea4a95d9.png)

$ |001\rangle$
![screen shot 2019-02-07 at 2 56 57 pm](https://user-images.githubusercontent.com/45162150/52393253-a88b6e00-2ae8-11e9-8be1-728317a884c1.png)

$ |010\rangle$
![screen shot 2019-02-07 at 2 58 23 pm](https://user-images.githubusercontent.com/45162150/52393298-d83a7600-2ae8-11e9-9c15-16675984831f.png)

$ |011\rangle$
![screen shot 2019-02-07 at 2 59 17 pm](https://user-images.githubusercontent.com/45162150/52393339-f7d19e80-2ae8-11e9-898b-10834c0a03d1.png)

$ |100\rangle$
![screen shot 2019-02-07 at 3 00 35 pm](https://user-images.githubusercontent.com/45162150/52393383-22235c00-2ae9-11e9-8d4e-da83903b79e5.png)

$ |101\rangle$
![screen shot 2019-02-07 at 3 01 40 pm](https://user-images.githubusercontent.com/45162150/52393419-4aab5600-2ae9-11e9-84a0-c5e028395ac8.png)

$ |110\rangle$
![screen shot 2019-02-07 at 3 02 22 pm](https://user-images.githubusercontent.com/45162150/52393442-6151ad00-2ae9-11e9-85cf-833667d2c3e0.png)

$ |111\rangle$
![screen shot 2019-02-07 at 3 02 55 pm](https://user-images.githubusercontent.com/45162150/52393459-79c1c780-2ae9-11e9-8dc0-b845e6f99e7e.png)　

In [64]:
#必要なモジュールのインポート
from qulacs import QuantumState,QuantumCircuit
from qulacs.gate import CZ,to_matrix_gate

#QulacsにはCCZゲートはないので自分で定義する。
def CCZ(q1,q2,q3):
    cz_gate = CZ(q2,q3)
    #CZゲートは特殊な高速化が施されているので、これを除いて他の処理を加えられるようにする。今回はコントロールビットを１つ加える。
    cz_mat_gate = to_matrix_gate(cz_gate)
    # 1st-qubitが1の場合だけゲートを作用させる
    control_index1 = q1
    control_with_value1 = 1
    #cz_mat_gateにコントロールビット(q1)を加える。
    cz_mat_gate.add_control_qubit(control_index1, control_with_value1)
    #新しいcz_mat_gateを出力する。
    return(cz_mat_gate)

#必要な量子ビット数
m = 3

#3量子ビットの量子状態を定義
state3 = QuantumState(m)

#状態を初期化する
state3.set_zero_state()

#3量子ビットの量子回路を構築する
circuit = QuantumCircuit(m)

#重ね合わせ状態を生成する
for i in range(m):
    circuit.add_H_gate(i)

#|111>の確率振幅を反転させる
circuit.add_gate(CCZ(0,1,2))

#拡散変換を施す。
for i in range(m):
    circuit.add_H_gate(i)
    circuit.add_X_gate(i)
    
circuit.add_gate(CCZ(0,1,2))

for i in range(m):
    circuit.add_X_gate(i)
    circuit.add_H_gate(i)

#これらの変更を回路及び状態に反映させる
circuit.update_quantum_state(state3)

#状態ベクトル(確率振幅)を出力する
print(state3.get_vector())    

[-0.1767767 +0.j -0.1767767 +0.j -0.1767767 +0.j -0.1767767 +0.j
 -0.1767767 +0.j -0.1767767 +0.j -0.1767767 +0.j -0.88388348+0.j]


よって$|111\rangle$の確率振幅の絶対値は約0.88(得られる確率が約78%)になるので、$|111\rangle$が得られる確率が1番高くなる。

参考文献
[1] M. A. Nielsen and I. L. Chuang, "Quantum Computation and Quantum Information 10th Anniversary Edition", University Printing House, 2018, 250p

[2] M. A. Nielsen and I. L. Chuang, "Quantum Computation and Quantum Information 10th Anniversary Edition", University Printing House, 2018, 248p

[3] E. Borbely,"Grover searching algorithm -arXiv",https://arxiv.org/pdf/0705.4171.pdf (参照 2019-02-27)

[4] 中山茂　「Python 量子プログラミング入門」Gaia教育シリーズ8 2018年　101-102p

[5] 「グローバーのアルゴリズム -Wikipedia」、https://ja.wikipedia.org/wiki/グローバーのアルゴリズム (参照 2019-02-27)